In [1]:
%load_ext autoreload
%autoreload 2
%pdb
import logging
logging.basicConfig()
#logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().setLevel(logging.INFO)

Automatic pdb calling has been turned ON


In [2]:
import sys
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
sys.path.insert(0, '/home/ubuntu/Documents/infembed/data')
from data._core.spotcheck import get_spotcheck_dataloader, get_blindspots_df
sys.path.insert(0, '/home/ubuntu/Documents/infembed/models')
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/')
# sys.path.insert(0, )
from infembed.embedder._core.fast_kfac_embedder import FastKFACEmbedder
import torchvision
from torch.utils.data import Subset, DataLoader, default_collate, Dataset
from torchvision.models import ResNet18_Weights, resnet18
import torch.nn as nn
from infembed.clusterer._core.sklearn_clusterer import SklearnClusterer
from infembed.clusterer._core.rule_clusterer import RuleClusterer
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd
import torch
from typing import List
from infembed.utils.common import Data
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable
from torch import Tensor
from typing import Tuple
from models._utils.common import init_linear
from lightning.pytorch.callbacks import EarlyStopping
import lightning.pytorch as pl
import lightning as L
import torch.nn.functional as F
import numpy as np

#### generate data
For each class, generate clusters with a specified training and test label.  procedurally, define, for each cluster, training label, test label, feature generator function

In [3]:
# struct to define clusters
@dataclass
class Config:
    label_generator: Callable
    feature_generator: Callable
    num: int
    blindspot: int = 0

In [4]:
# define possible `feature_generator`
@dataclass
class GaussianFeatureGenerator:
    mean: Tensor
    std: float = 1.

    def __call__(self):
        return torch.normal(self.mean, self.std)

In [5]:
# define possible `label_generator`
@dataclass
class CategoricalLabelGenerator:
    probs: List[float]

    def __call__(self):
        #import pdb
        #pdb.set_trace()
        return torch.tensor(np.random.choice(len(self.probs), p=self.probs))

In [6]:
# define the configs
if False:
    std = 0.5
    num_train = 1000
    num_val = 100
    num_test = 1000
    D = 2
    C = 2

    train_configs = [
        #Config(CategoricalLabelGenerator([0,1]), GaussianFeatureGenerator(Tensor([0, 0]), std), num_train),
        #Config(CategoricalLabelGenerator([0,1]), GaussianFeatureGenerator(Tensor([2, 0]), std), num_train),
        Config(CategoricalLabelGenerator([0,1]), GaussianFeatureGenerator(Tensor([0, 1]), std), num_train),
        Config(CategoricalLabelGenerator([1,0]), GaussianFeatureGenerator(Tensor([2, 1]), std), num_train),
        Config(CategoricalLabelGenerator([1,0]), GaussianFeatureGenerator(Tensor([1, 0]), std), num_train),
        Config(CategoricalLabelGenerator([1,0]), GaussianFeatureGenerator(Tensor([3, 0]), std), num_train),
        Config(CategoricalLabelGenerator([1,0]), GaussianFeatureGenerator(Tensor([1, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0,1]), GaussianFeatureGenerator(Tensor([3, 1]), std), num_train),
    ]

    val_configs = train_configs

    test_configs = [
        #Config(CategoricalLabelGenerator([0,1]), GaussianFeatureGenerator(Tensor([0, 0]), std), num_train),
        #Config(CategoricalLabelGenerator([0,1]), GaussianFeatureGenerator(Tensor([2, 0]), std), num_train),
        Config(CategoricalLabelGenerator([0,1]), GaussianFeatureGenerator(Tensor([0, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0.3,0.7]), GaussianFeatureGenerator(Tensor([2, 1]), std), num_train, 1),
        Config(CategoricalLabelGenerator([1,0]), GaussianFeatureGenerator(Tensor([1, 0]), std), num_train),
        Config(CategoricalLabelGenerator([1,0]), GaussianFeatureGenerator(Tensor([3, 0]), std), num_train),
        Config(CategoricalLabelGenerator([1,0]), GaussianFeatureGenerator(Tensor([1, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0.7,0.3]), GaussianFeatureGenerator(Tensor([3, 1]), std), num_train, 1),
    ]

if True:
    std = 0.5
    num_train = 1000
    num_val = 100
    num_test = 1000
    D = 2
    C = 3

    # for each of 3 classes, 3 clusters, 2 of them are mis-labeled to other class

    train_configs = [
        Config(CategoricalLabelGenerator([1,0,0]), GaussianFeatureGenerator(Tensor([0, 0]), std), num_train),
        Config(CategoricalLabelGenerator([1,0,0]), GaussianFeatureGenerator(Tensor([2, 0]), std), num_train),
        Config(CategoricalLabelGenerator([0.5,0.25,0.25]), GaussianFeatureGenerator(Tensor([0, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0,1,0]), GaussianFeatureGenerator(Tensor([2, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0.25,0.5,0.25]), GaussianFeatureGenerator(Tensor([1, 0]), std), num_train),
        Config(CategoricalLabelGenerator([0,1,0]), GaussianFeatureGenerator(Tensor([3, 0]), std), num_train),
        Config(CategoricalLabelGenerator([0,0,1]), GaussianFeatureGenerator(Tensor([1, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0.25,0.25,0.5]), GaussianFeatureGenerator(Tensor([3, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0,0,1]), GaussianFeatureGenerator(Tensor([4, 1]), std), num_train),
    ]

    val_configs = train_configs

    test_configs = [
        Config(CategoricalLabelGenerator([1,0,0]), GaussianFeatureGenerator(Tensor([0, 0]), std), num_train),
        Config(CategoricalLabelGenerator([1,0,0]), GaussianFeatureGenerator(Tensor([2, 0]), std), num_train),
        Config(CategoricalLabelGenerator([1,0,0]), GaussianFeatureGenerator(Tensor([0, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0,1,0]), GaussianFeatureGenerator(Tensor([2, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0,1,0]), GaussianFeatureGenerator(Tensor([1, 0]), std), num_train),
        Config(CategoricalLabelGenerator([0,1,0]), GaussianFeatureGenerator(Tensor([3, 0]), std), num_train),
        Config(CategoricalLabelGenerator([0,0,1]), GaussianFeatureGenerator(Tensor([1, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0,0,1]), GaussianFeatureGenerator(Tensor([3, 1]), std), num_train),
        Config(CategoricalLabelGenerator([0,0,1]), GaussianFeatureGenerator(Tensor([4, 1]), std), num_train),
    ]

In [7]:
# generate datasets and metadata using configs
def generate_data(configs: List[Config]):
    xs, ys = [], []
    metadatas = []
    for config in configs:

        def generate_data_cluster(config: Config):
            num = config.num
            return (
                torch.stack([config.feature_generator() for _ in range(num)]),
                torch.Tensor([config.label_generator() for _ in range(num)]).long(),
            ), pd.DataFrame({"blindspot_dummy": (torch.ones(num) * config.blindspot).long()})

        (x, y), metadata = generate_data_cluster(config)
        xs.append(x)
        ys.append(y)
        metadatas.append(metadata)

    metadata = pd.concat(metadatas, axis=0)
    metadata.index = list(range(len(metadata)))

    return (torch.cat(xs, dim=0), torch.cat(ys, dim=0)), metadata


class DatasetFromTuple(Dataset):
    def __init__(self, *tup: Tuple[Tensor]):
        self.tup = tup

    def __getitem__(self, i: int):
        return tuple([t[i] for t in self.tup])

    def __len__(self):
        return len(next(iter(self.tup)))


(train_xs, train_ys), train_metadata = generate_data(train_configs)
(val_xs, val_ys), val_metadata = generate_data(val_configs)
(test_xs, test_ys), test_metadata = generate_data(test_configs)

train_dataset = DatasetFromTuple(train_xs, train_ys)
val_dataset = DatasetFromTuple(val_xs, val_ys)
test_dataset = DatasetFromTuple(test_xs, test_ys)

In [8]:
# create dataloaders
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#### create model

In [9]:
class MLP(pl.LightningModule):
    def __init__(self, dims: List[int], lr: float):
        super().__init__()
        self.linears = nn.ModuleList(
            [
                nn.Linear(dim_in, dim_out)
                for (dim_in, dim_out) in zip(dims[:-1], dims[1:])
            ]
        )
        self.lr = lr
        self.apply(init_linear)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def forward(self, x):
        for l in self.linears[:-1]:
            x = l(x)
            x = torch.sigmoid(x)
        return self.linears[-1](x)

    def _step(self, batch, batch_idx):
        # run forward
        x, y = batch
        y_hat = self.forward(x)
        loss = F.cross_entropy(y_hat, y, reduction="mean")
        return {"loss": loss}

    def training_step(self, batch, batch_idx):
        d = self._step(batch, batch_idx)
        self.log_dict(
            {f"train_{key}": val for (key, val) in d.items() if key[0] != "_"}
        )
        return d

    def validation_step(self, batch, batch_idx):
        d = self._step(batch, batch_idx)
        self.log_dict(
            {f"validation_{key}": val for (key, val) in d.items() if key[0] != "_"}
        )
        return d

    def prediction_step(self, batch, batch_idx):
        d = self._step(batch, batch_idx)
        self.log_dict(
            {f"prediction_{key}": val for (key, val) in d.items() if key[0] != "_"}
        )
        return d
    

model = MLP([D, 100, 100, 100, C], 1e-3)
print(f"total parameters: {sum(p.numel() for p in model.parameters())}")
model

total parameters: 20803


MLP(
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1-2): 2 x Linear(in_features=100, out_features=100, bias=True)
    (3): Linear(in_features=100, out_features=3, bias=True)
  )
)

#### define trainer

In [10]:
trainer = L.Trainer(
    callbacks=[
        EarlyStopping(
            patience=3,
            monitor='validation_loss',
            mode='min',
            verbose=True,
            min_delta=0.1,
            ),
    ],
    max_epochs=10,
)

INFO: Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning.pytorch.utilities.rank_zero:Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU av

#### train

In [11]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
INFO: 
  | Name    | Type       | Params
---------------------------------------
0 | linears | ModuleList | 20.8 K
---------------------------------------
20.8 K    Trainable params
0         Non-trainable params
20.8 K    Total params
0.083     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type       | Params
---------------------------------------
0 | linears | ModuleList | 20.8 K
---------------------------------------
20.8 K    Trainable params
0         Non-trainable params
20.8 K    Total params
0.083     Total estimated model params size (MB)


Sanity Checking: |                                                                                            …

/home/ubuntu/miniconda3/envs/test2/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/ubuntu/miniconda3/envs/test2/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/home/ubuntu/miniconda3/envs/test2/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: |                                                                                                 …

Validation: |                                                                                                 …

INFO: Monitored metric validation_loss did not improve in the last 3 records. Best score: 0.932. Signaling Trainer to stop.
INFO:lightning.pytorch.callbacks.early_stopping:Monitored metric validation_loss did not improve in the last 3 records. Best score: 0.932. Signaling Trainer to stop.


#### add predictions and labels to metadata

In [12]:
def _get_predictions_and_labels(_model, dataloader):
    dfs = []
    for batch in tqdm(dataloader):
        prediction_prob = (
            torch.nn.functional.softmax(_model(*batch[:-1]), dim=1)
            .detach()
            .to(device="cpu")
        )
        prediction_label = torch.argmax(prediction_prob, dim=1).to(device="cpu")
        label = batch[-1].to(
            device="cpu"
        )  # assuming batch is a tensor.  if not, can check
        # import pdb
        # pdb.set_trace()
        dfs.append(
            pd.DataFrame(
                {
                    "prediction_label": prediction_label,
                    "label": label,
                    "prediction_prob": list(prediction_prob.numpy()),
                    "blindspot": (prediction_label != label).float(),
                }
            )
        )
    df = pd.concat(dfs, axis=0)
    df.index = list(range(len(df)))
    return df

test_predictions = _get_predictions_and_labels(model, test_dataloader)
test_metadata = pd.concat([test_metadata, test_predictions], axis=1)

 30%|██████████████████████████████████████████████████▎                                                                                                                  | 86/282 [00:00<00:00, 858.24it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 282/282 [00:00<00:00, 855.44it/s]


#### define embedder

#### 

In [56]:
from infembed.embedder._core.arnoldi_embedder import ArnoldiEmbedder
from infembed.embedder._core.dim_reduct_embedder import PCAEmbedder
from infembed.embedder._core.gradient_embedder import GradientEmbedder

if True:
    layers = ['linears.0','linears.1']
if False:
    layers = None

if True:
    projection_dim = 2

if False:
    embedder = FastKFACEmbedder(
        layers=layers,
        loss_fn=nn.CrossEntropyLoss(reduction='mean'),
        sample_wise_grads_per_batch=True,
        projection_dim=projection_dim,
        show_progress=True,
        model=model,
    )

if False:
    embedder = ArnoldiEmbedder(
        layers=layers,
        loss_fn=nn.CrossEntropyLoss(reduction='mean'),
        sample_wise_grads_per_batch=True,
        projection_dim=projection_dim,
        show_progress=True,
        model=model,
        arnoldi_dim=20,
        vhp='hvp',
    )
if True:
    embedder = ArnoldiEmbedder(
        layers=layers,
        loss_fn=nn.CrossEntropyLoss(reduction='mean'),
        sample_wise_grads_per_batch=True,
        projection_dim=projection_dim,
        show_progress=True,
        model=model,
        arnoldi_dim=20,
        vhp='vhp',
    )
if False:
    embedder = ArnoldiEmbedder(
        layers=layers,
        loss_fn=nn.CrossEntropyLoss(reduction='mean'),
        sample_wise_grads_per_batch=True,
        projection_dim=projection_dim,
        show_progress=True,
        model=model,
        arnoldi_dim=20,
        vhp='manual_hvp',
    )
if False:
    embedder = ArnoldiEmbedder(
        layers=layers,
        loss_fn=nn.CrossEntropyLoss(reduction='mean'),
        sample_wise_grads_per_batch=True,
        projection_dim=projection_dim,
        show_progress=True,
        model=model,
        arnoldi_dim=20,
        vhp='manual_revrev',
    )
if False:
    embedder = PCAEmbedder(
        base_embedder=GradientEmbedder(
            layers=layers,
            loss_fn=nn.CrossEntropyLoss(reduction='mean'),
            sample_wise_grads_per_batch=True,
            show_progress=True,
            model=model,
        ),
        incremental_pca_kwargs={'n_components': projection_dim},
    )

#### get embeddings

In [57]:
%timeit -r 2 embeddings = embedder.fit(train_dataloader)
%timeit -r 2 embedder.predict(test_dataloader)
embeddings.shape

INFO:root:start arnoldi iteration
INFO:root:start `_parameter_arnoldi`
Running Arnoldi Iteration for step:   0%|                                                                                                                                            | 0/20 [00:00<?, ?it/s]INFO:root:arnoldi iteration step 1


Running Arnoldi Iteration for step: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:17<00:00,  1.17it/s]
INFO:root:start `_parameter_distill`
INFO:root:start arnoldi iteration
INFO:root:start `_parameter_arnoldi`
Running Arnoldi Iteration for step: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:18<00:00,  1.06it/s]
INFO:root:start `_parameter_distill`
INFO:root:start arnoldi iteration
INFO:root:start `_parameter_arnoldi`
Running Arnoldi Iteration for step: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:17<00:00,  1.15it/s]
INFO:root:start `_parameter_distill`


18.8 s ± 431 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


Using ArnoldiEmbedder to compute embeddings. Processing batch:   0%|          | 0/282 [00:00<?, ?it/s]

INFO:root:compute embeddings


...

INFO:root:compute embeddings


Using ArnoldiEmbedder to compute embeddings. Processing batch:   0%|          | 0/282 [00:00<?, ?it/s]

INFO:root:compute embeddings


587 ms ± 318 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)


torch.Size([9000, 2])

#### define clusterer

In [22]:
if False:
    n_clusters = 20
if False:
    n_clusters = 10
if False:
    n_clusters = 5
if True:
    n_clusters = 15
clusterer = SklearnClusterer(sklearn_clusterer=KMeans(n_clusters=n_clusters))

#### create `Data`

In [23]:
data = Data(
    embeddings=embeddings,
    metadata=test_metadata,
)

#### cluster

In [24]:
clusters = clusterer.fit_predict(data)

#### how to display

In [25]:
from infembed.visualization._core.common import (
    PerClusterDisplayer,
    DisplayAccuracy,
    SingleClusterDisplayer,
    DisplayCounts,
    ClusterAUCDisplayer,
)


displayers = [
    ClusterAUCDisplayer('blindspot'),
    PerClusterDisplayer(
        [
            DisplayAccuracy(prediction_col="prediction_label", label_col="label"),
            DisplayCounts(['blindspot', 'label'])
            ]
    )
]

#### display

In [26]:
for displayer in displayers:
    displayer(clusters, data)

AUC for blindspot: 0.9537726000810658
cluster #0
accuracy: 0.32 (231/718)
blindspot: {0.0: 231, 1.0: 487}
label: {1: 718}
cluster #1
accuracy: 1.00 (834/834)
blindspot: {0.0: 834}
label: {2: 834}
cluster #2
accuracy: 0.84 (332/395)
blindspot: {0.0: 332, 1.0: 63}
label: {0: 395}
cluster #3
accuracy: 0.95 (757/800)
blindspot: {0.0: 757, 1.0: 43}
label: {1: 800}
cluster #4
accuracy: 0.16 (94/600)
blindspot: {0.0: 94, 1.0: 506}
label: {0: 600}
cluster #5
accuracy: 1.00 (931/931)
blindspot: {0.0: 931}
label: {0: 931}
cluster #6
accuracy: 0.05 (13/253)
blindspot: {0.0: 13, 1.0: 240}
label: {2: 253}
cluster #7
accuracy: 0.10 (82/802)
blindspot: {0.0: 82, 1.0: 720}
label: {1: 802}
cluster #8
accuracy: 0.36 (83/232)
blindspot: {0.0: 83, 1.0: 149}
label: {2: 232}
cluster #9
accuracy: 0.92 (846/921)
blindspot: {0.0: 846, 1.0: 75}
label: {2: 921}
cluster #10
accuracy: 0.01 (3/478)
blindspot: {0.0: 3, 1.0: 475}
label: {0: 478}
cluster #11
accuracy: 0.92 (624/680)
blindspot: {0.0: 624, 1.0: 56}
labe

### do everything, but for different embedders, clusterers